In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from src.cyc_gbm import CycGBM
from scipy.optimize import minimize

In [2]:
n = 1000
rng = np.random.default_rng(seed=10)
X0 = np.arange(0, n) / n
X1 = np.arange(0, n) / n
rng.shuffle(X1)
mu = np.exp(1 * (X0 > 0.3 * n) + 0.5 * (X1 > 0.5 * n))
v = np.exp(1 + 1 * X0 - 3 * np.abs(X1))

X = np.stack([X0, X1]).T
alpha = mu * (1 + v)
beta = v + 2
y0 = rng.beta(alpha, beta)
y = y0 / (1 - y0)

max_depth = 2
min_samples_leaf = 20
eps = [0.1, 0.1]
kappa = [20, 100]

In [3]:
gbm = CycGBM(
    kappa=kappa,
    eps=eps,
    max_depth=max_depth,
    min_samples_leaf=min_samples_leaf,
    dist="beta_prime",
)
gbm.fit(X, y)
z_hat_new = gbm.predict(X)

In [4]:
# Conversion to old structure
zOpt = gbm.z0
BOpts = kappa
BOpt = max(kappa)
d = 2
epsilon = eps

In [5]:
# Distribution dependent
from scipy.special import polygamma as pg
from scipy.special import loggamma as lg


# Loss function
def lossEval(z, y):
    return gbm.dist.loss(z=z, y=y)


# Gradients
def grad0(z, y):
    return gbm.dist.grad(z=z, y=y, j=0)


def grad1(z, y):
    return gbm.dist.grad(z=z, y=y, j=1)


# Gradient list
grads = [grad0, grad1]

In [6]:
# Setup
from sklearn.tree import DecisionTreeRegressor as regressionTree


# Tree fitting function
def trainTree(z, y, x, grad, max_depth, min_samples_leaf):
    tree = regressionTree(max_depth=max_depth, min_samples_leaf=min_samples_leaf)
    g = grad(z, y)
    tree.fit(x, -g)
    return tree


# Dimension indicator for score adding
e = [np.stack([[0]] * j + [[1]] + [[0]] * (d - j - 1)) for j in range(0, d)]


# Local tree training function
def trainThisTree(z, y, x, grad, max_depth, min_samples_leaf):
    return trainTree(
        z=z, y=y, x=x, grad=grad, max_depth=max_depth, min_samples_leaf=min_samples_leaf
    )

In [7]:
from src.distributions import opt_step_numeric

In [8]:
# Training and validation data
xTrain, yTrain = X, y
xValid, yValid = X, y
nTrain, nValid = len(yTrain), len(yValid)

# Initiate score
zTrain = zOpt.repeat(nTrain).reshape((d, nTrain))
zValid = zOpt.repeat(nValid).reshape((d, nValid))

trees = [[[]] * BOpts[0], [[]] * BOpts[1]]

# Build trees
for b in range(0, BOpt):
    # For all parameter dimensions
    for j in range(0, d):
        if b < BOpts[j]:
            # Fit a tree
            tree = trainThisTree(
                zTrain,
                yTrain,
                xTrain,
                grads[j],
                max_depth=max_depth,
                min_samples_leaf=min_samples_leaf,
            )

            # Score updates
            zTrainPlus = tree.predict(xTrain)
            zValidPlus = tree.predict(xValid)

            # Look at all unique node values
            nodeValues = np.unique(zTrainPlus)
            for nodeValue in nodeValues:
                # Indices with this node value
                indicesTrain = zTrainPlus == nodeValue
                indicesValid = zValidPlus == nodeValue
                # Optimize step size
                gammaOpt = minimize(
                    lambda gamma: lossEval(
                        z=zTrain[:, indicesTrain] + e[j] * gamma, y=yTrain[indicesTrain]
                    ).sum(),
                    x0=nodeValue,
                )["x"][0]
                # Manipulate node values
                zTrainPlus[indicesTrain] = gammaOpt
                zValidPlus[indicesValid] = gammaOpt
                tree.tree_.value[tree.tree_.value == nodeValue] = gammaOpt

            # Update score
            zTrain[j] += epsilon[j] * zTrainPlus
            zValid[j] += epsilon[j] * zValidPlus

            # Update trees
            trees[j][b] = tree

In [9]:
# Assume two dimensions
z_hat_old = zOpt.repeat(nTrain).reshape((d, nTrain))
for j in [0, 1]:
    z_hat_j = epsilon[j] * sum([tree.predict(X) for tree in trees[j]])
    z_hat_old[j] += z_hat_j

In [10]:
print(f"old model loss: {gbm.dist.loss(z_hat_old,y).sum().round(2)}")
print(f"new model loss: {gbm.dist.loss(z_hat_new,y).sum().round(2)}")

old model loss: 121.23
new model loss: 121.23


In [15]:
gbm.dist.loss(z_hat_old, y).sum()

121.22775641886105

In [11]:
gbm.trees[0][-1].tree_.node_count

5

In [12]:
trees[0][-1].tree_.node_count

5

In [13]:
e[1].shape

(2, 1)

In [14]:
np.array([[0, 1]]).T.shape

(2, 1)